In [ ]:
from sklearn.preprocessing import StandardScaler
from utils.ds_clean import clean_ds
from tensorflow.keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt




In [ ]:
df_tets = clean_ds(pd.read_csv('data/aapl_5m_test.csv'))
model_buy = load_model('deep_learning/model_buy_rnn.keras')
model_sell = load_model('deep_learning/model_sell_rnn.keras')

close_data_test = df_tets[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']].dropna()
cl_dat_test = close_data_test.copy()

x_buy_test = cl_dat_test.drop(columns=['Timestamp','Y_BUY']).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_buy_test)
X_reshaped = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

close_data_test['Y_BUY_PRED_RNN'] = (model_buy.predict(X_reshaped)) > 0.5
close_data_test = close_data_test.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3','X_t-4' ,'X_t-5'])
close_data_test['Y_SELL_PRED_RNN'] = (model_sell.predict(X_reshaped)) > 0.5

#convertir a booleano


In [ ]:
close_data_test

Best buy/sell overall strategy: rnn_strategy with value: 5635457.340371052
Best buy/sell strategy parameters: {'stop_loss': 0.03202113504755525, 'take_profit': 0.04768956604374807, 'n_shares': 101}

In [ ]:
cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]
stop_loss = 0.013457462530738862
take_profit = 0.04031579058294208
n_shares = 158

class Operation:
    def __init__(self, operation_type, bought_at, timestamp, n_shares, stop_loss, take_profit):
        self.operation_type = operation_type
        self.bought_at = bought_at
        self.timestamp = timestamp
        self.n_shares = n_shares
        # self.sold_at = None
        self.stop_loss = stop_loss
        self.take_profit = take_profit

for i, row in close_data_test.iterrows():
    # Close Operations
    temp_operations = []
    for op in active_operations:
        if op.operation_type == 'Long':
            if op.stop_loss > row.Close:
                cash += row.Close * op.n_shares * (1 - com)
            elif op.take_profit < row.Close:
                cash += row.Close * op.n_shares * (1 - com)
            else:
                temp_operations.append(op)
        elif op.operation_type == 'Short':
            if op.stop_loss < row.Close:
                cash -= row.Close * op.n_shares * (1 + com)
            elif op.take_profit > row.Close:
                cash -= row.Close * op.n_shares * (1 + com)
            else:
                temp_operations.append(op)
    active_operations = temp_operations

    # Open Operations
    if row.Y_BUY_PRED_RNN:
        n_shares = n_shares
        stop_loss = row.Close * stop_loss
        take_profit = row.Close * (1 + take_profit)
        active_operations.append(
            Operation('Long', row.Close, row.Timestamp, n_shares, stop_loss, take_profit))
        cash -= row.Close * n_shares * (1 + com)
    elif row.Y_SELL_PRED_RNN:
        n_shares = n_shares
        stop_loss = row.Close * (1 + stop_loss)
        take_profit = row.Close * take_profit
        active_operations.append(
            Operation('Short', row.Close, row.Timestamp, n_shares, stop_loss, take_profit))
        cash += row.Close * n_shares * (1 - com)

    total_value = len(active_operations) * row.Close * n_shares
    strategy_value.append(cash + total_value)
    
plt.figure(figsize=(12, 6))
plt.plot(strategy_value)
plt.title('Valor de la estrategia RNN')
plt.xlabel('Período')
plt.ylabel('Valor')
plt.show()
print("Valor de la estrategia RNN: ", strategy_value[-1])